In [1]:
import malariagen_data
import allel
import matplotlib.pyplot as plt
import pandas as pd
import plotly.express as px
import numpy as np
import seaborn as sns

In [2]:
ag3 = malariagen_data.Ag3(
    results_cache="/Users/dennistpw/Projects/drc_genomic_surveillance/data/",
    pre=True
)
ag3

<MalariaGEN Ag3 API client>
Storage URL             : gs://vo_agam_release/
Data releases available : 3.0, 3.1, 3.10, 3.11, 3.2, 3.3, 3.4, 3.5, 3.6, 3.7, 3.8, 3.9
Results cache           : /Users/dennistpw/Projects/drc_genomic_surveillance/data
Cohorts analysis        : 20240717
AIM analysis            : 20220528
Site filters analysis   : dt_20200416
Software version        : malariagen_data 0.0.0.post782+5e502bf
Client location         : Queensland, Australia
---
Please note that data are subject to terms of use,
for more information see https://www.malariagen.net/data
or contact support@malariagen.net. For API documentation see 
https://malariagen.github.io/malariagen-data-python/v0.0.0.post782+5e502bf/Ag3.html

In [9]:
#Define all the sample sets we want to include and exclude based on Poppy's analyses
drc_sample_sets = [
    '1264-VO-CD-WATSENGA-VMF00161',
    '1264-VO-CD-WATSENGA-VMF00164',
]
ref_sample_sets = [
    # 'AG1000G-CD',
    'AG1000G-CF',
    'AG1000G-TZ',
    'AG1000G-UG',
#    'AG1000G-GA-A',
#    'AG1000G-MZ',
    'AG1000G-CM-A',
    'AG1000G-BF-A',
    '1273-VO-ZM-MULEBA-VMF00176',
]
sample_sets = drc_sample_sets + ref_sample_sets

locations_blacklist = [
    'Mayos',
    'Daiguene',
    'Gbadolite',
    'Muheza',
]

samples_blacklist = [
    "AB0203-C",
    "AB0177-C",
    "AN0085-C",
    "AN0107-C",
    "AC0223-C",
    "AC0240-C",
    "AB0207-C",
    "AB0158-Cx",
]

drc_samples_blacklist = ['VBS48708-6367STDY9888400', 'VBS48725-6367STDY9888417']

cat_ordered = [
    'Burkina Faso',
    'Cameroon',
    'Central African Republic',
    'Uganda',
    'Tanzania',
    'Zambia',
    'DRC Far North',
    'DRC South East',
    'DRC South West',
    'DRC Forest'
]

In [12]:
# Now define sample analysis pop from metadata

drc_metadata = ag3.sample_metadata(sample_query=(
        "taxon == 'gambiae' and "
        f"sample_set in {sample_sets} and "
        f"location not in {locations_blacklist} and "
        f"sample_id not in {samples_blacklist + drc_samples_blacklist}"
    ))

# add coloring category
drc_metadata.loc[:, "group"] = drc_metadata["country"].copy()
drc_metadata.loc[(drc_metadata["admin1_name"] == "Nord-Ubangi"), "group"] = "DRC Far North"
drc_metadata.loc[(drc_metadata["admin1_name"] == "Upper Uele"), "group"] = "DRC Far North"
drc_metadata.loc[(drc_metadata["admin1_name"] == "Kongo-Central"), "group"] = "DRC South West"
drc_metadata.loc[(drc_metadata["admin1_name"] == "Kinshasa"), "group"] = "DRC South West"
drc_metadata.loc[(drc_metadata["admin1_name"] == "Mai-Ndombe"), "group"] = "DRC South West"
drc_metadata.loc[(drc_metadata["admin1_name"] == "Sankuru"), "group"] = "DRC South West"
drc_metadata.loc[(drc_metadata["admin1_name"] == "South Kivu"), "group"] = "DRC South East"
drc_metadata.loc[(drc_metadata["admin1_name"] == "Central Kasai"), "group"] = "DRC South East"
drc_metadata.loc[(drc_metadata["admin1_name"] == "Tanganyika"), "group"] = "DRC South East"
drc_metadata.loc[(drc_metadata["admin1_name"] == "Haut-Katanga"), "group"] = "DRC South East"
drc_metadata.loc[(drc_metadata["admin1_name"] == "Tshopo"), "group"] = "DRC Forest (Tshopo)"
drc_metadata.groupby("group").size()

group
Burkina Faso                 95
Cameroon                     95
Central African Republic     55
DRC Far North                46
DRC Forest (Tshopo)         118
DRC South East              320
DRC South West              187
Tanzania                     32
Uganda                      205
Zambia                      201
dtype: int64

In [41]:
selscan_list = []
for pop in drc_metadata.group.unique():
    sample_list = drc_metadata[drc_metadata.group == pop]['sample_id'].tolist()
    for chrom in ag3.contigs:
        h = ag3.h12_gwss(
               contig=chrom,
               sample_query=f'sample_id=={sample_list}',
               cohort_size=20,
               window_size=2000,
           )
        h = (*h, chrom)
        h = (*h, pop)
        selscan_list.append(h)
        


In [97]:
#set up some plotting variables
maxpos = []
for i in [0,1,2,3,4]:
    maxpos.append(selscan_list[i][0].max())
total_max = sum(maxpos)
column_widths = (maxpos / total_max)
col_var_levels = ag3.contigs

#import useful libs
from matplotlib.ticker import FuncFormatter
from matplotlib.gridspec import GridSpec


In [76]:
row_var_levels = drc_metadata.group.unique()
rowvars_reordered = ['Burkina Faso','Cameroon', 'Central African Republic','DRC Far North','DRC Forest (Tshopo)', 'DRC South East','Zambia','Tanzania','Uganda']

In [93]:
# Convert each tuple into a DataFrame, expanding shorter lists
dfs = []
for i, (pos, h12, chrom, pop) in enumerate(selscan_list):
    max_len = len(pos)  # Determine the length to expand to
    df = pd.DataFrame({
        'pos': pos,
        'h12': h12,
        'chrom': chrom,  # Repeat chromosome to match length
        'pop': pop       # Repeat population to match length
    })
    dfs.append(df)

# Combine all DataFrames into one
selscan_df = pd.concat(dfs, ignore_index=True)



In [98]:
col_var_levels

('2R', '2L', '3R', '3L', 'X')

In [ ]:
# Get the unique row and column variables
row_var_levels = drc_metadata.group.unique()

#set y labs to 2 dp
def format_func(value, tick_number):
    return f"{value:.2f}"

# Initialize the figure and GridSpec
fig = plt.figure(figsize=(sum(column_widths) * 25, len(row_var_levels) * 1.5))  # Plots half as high
gs = GridSpec(len(row_var_levels), len(col_var_levels) + 1, width_ratios=list(column_widths) + [0.1])

# Create the subplots
for row_idx, row_val in enumerate(rowvars_reordered):
    for col_idx, col_val in enumerate(col_var_levels):
        row_colour = pop_code_cols.get(row_val, 'black')  # Default color if not specified

        ax = fig.add_subplot(gs[row_idx, col_idx])
        subset = selscan_df[(selscan_df['pop'] == row_val) & (selscan_df['chrom'] == col_val)]
        sns.lineplot(data=subset, x='midpos', y='h1', ax=ax, linewidth=0.8, color=row_colour)

        ax.fill_between(x=subset['midpos'], y1=subset['h1'], color=row_colour, alpha=0.1)  # Adjust alpha for transparency if needed


        #Let's start by tinkering with the axes

        #set max on y to be max h1 for each pop - this emphasises the sweep signal but we must note in the legend!
        ax.set_ylim(0, 1)
        ax.yaxis.set_major_formatter(FuncFormatter(format_func))
        #set y max as ticks
        ax.set_yticks([0, 1])
        ax.set_ylabel("")
        plt.subplots_adjust(top=0.85) 

        #now we want to remove clutter as much as possible
        #despine
        ax.spines['top'].set_visible(False)
        ax.spines['right'].set_visible(False)
        ax.spines['left'].set_visible(False)
        ax.spines['bottom'].set_visible(False)

        #remove x axes except bottom one
        if row_idx < len(rowvars_reordered) - 1:
            ax.xaxis.set_tick_params(which='both', bottom=False, top=False, labelbottom=False)  # Hide x-axis ticks and labels

        # Remove y-axis for 2nd and 3rd columns
        if col_idx > 0:
            ax.set_ylabel('')
            ax.yaxis.set_visible(False)

        # Remove x-axis titles and replace with chromosome numbering
        ax.set_xlabel('')
        if row_idx == 10:
            if col_idx == 0:
                ax.set_xlabel("2R", fontsize=24, fontweight='bold')
            elif col_idx == 1:
                ax.set_xlabel("2L", fontsize=24, fontweight='bold')
            elif col_idx == 2:
                ax.set_xlabel("3R", fontsize=24, fontweight='bold')  
            elif col_idx == 3:
                ax.set_xlabel("3L", fontsize=24, fontweight='bold')
            elif col_idx == 4:
                ax.set_xlabel("X", fontsize=24, fontweight='bold')

        #add row (population) labels
        row_ax = fig.add_subplot(gs[row_idx, -1])
        row_ax.text(0.5, 0.5, row_val, va='center', ha='center', fontsize=18, rotation=0, transform=row_ax.transAxes)
        row_ax.axis('off')

        #now, let's annotate our plots

        # #add text annotations and lines for IR genes
        # if col_val == 'CM023248' and row_idx == 0:
        #     # Cyp6
        #     ax.fill_betweenx([cyp6_region[col_val]['y_min'], cyp6_region[col_val]['y_max']],
        #         cyp6_region[col_val]['x_min'], cyp6_region[col_val]['x_max'],
        #         color='gray', alpha=0.5)
        #     ax.text(cyp6_region[col_val]['x_min'], 1, "Cyp6", fontsize=18, rotation=90)
        #     # Ace1
        #     ax.text(ace1_region[col_val]['x_min'], 1, "Ace1", fontsize=18, rotation=90)
        #     ax.fill_betweenx([ace1_region[col_val]['y_min'], ace1_region[col_val]['y_max']],
        #         ace1_region[col_val]['x_min'], ace1_region[col_val]['x_max'],
        #         color='gray', alpha=0.5)

        # elif col_val == 'CM023249' and row_idx == 0:
        #     #Vgsc
        #     ax.fill_betweenx([vgsc_region[col_val]['y_min'], vgsc_region[col_val]['y_max']],
        #         vgsc_region[col_val]['x_min'], vgsc_region[col_val]['x_max'],
        #         color='gray', alpha=0.5)
        #     ax.text(vgsc_region[col_val]['x_min'], 1, "Vgsc", fontsize=18, rotation=90)

        #     #Gste
        #     ax.fill_betweenx([gste_region[col_val]['y_min'], gste_region[col_val]['y_max']],
        #         gste_region[col_val]['x_min'], gste_region[col_val]['x_max'],
        #         color='gray', alpha=0.5)
        #     ax.text(gste_region[col_val]['x_min'], 1, "Gste", fontsize=18, rotation=90)

        #     #Rdl
        #     ax.fill_betweenx([rdl_region[col_val]['y_min'], rdl_region[col_val]['y_max']],
        #         rdl_region[col_val]['x_min'], rdl_region[col_val]['x_max'],
        #         color='gray', alpha=0.5)
        #     ax.text(rdl_region[col_val]['x_min'],1, "Rdl", fontsize=18, rotation=90)

        #     #Coeae
        #     ax.fill_betweenx([carboxyl_cluster[col_val]['y_min'], carboxyl_cluster[col_val]['y_max']],
        #                     carboxyl_cluster[col_val]['x_min'], carboxyl_cluster[col_val]['x_max'],
        #                     color='gray', alpha=0.5)
        #     ax.text(carboxyl_cluster[col_val]['x_min'], 1, "Coeae", fontsize=18, rotation=90)

        #     #CoeJH
        #     ax.fill_betweenx([COEJHE5E_region[col_val]['y_min'], COEJHE5E_region[col_val]['y_max']],
        #                     COEJHE5E_region[col_val]['x_min'], COEJHE5E_region[col_val]['x_max'],
        #                     color='gray', alpha=0.5)
        #     ax.text(COEJHE5E_region[col_val]['x_min'], 1, "Coejh", fontsize=18, rotation=90)

        # elif col_val == 'CM023250' and row_idx == 0:
        #     #Cyp9k1
        #     ax.fill_betweenx([cyp9_region[col_val]['y_min'], cyp9_region[col_val]['y_max']],
        #         cyp9_region[col_val]['x_min'], cyp9_region[col_val]['x_max'],
        #         color='gray', alpha=0.5)
        #     ax.text(cyp9_region[col_val]['x_min'], 1, "Cyp9k1", fontsize=18, rotation=90)

        #     #Diagk
        #     ax.fill_betweenx([diagk[col_val]['y_min'], diagk[col_val]['y_max']],
        #         diagk[col_val]['x_min'], diagk[col_val]['x_max'],
        #         color='gray', alpha=0.5)
        #     ax.text(diagk[col_val]['x_min'], 1, "Diagk", fontsize=18, rotation=90)

#Add master Y labels
fig.text(0.08, 0.5, 'H1', va='center', rotation='vertical', fontweight='bold', fontsize=18)

plt.subplots_adjust(wspace=0, hspace=0.2)  # Reduce column and row padding

fig.savefig('../figures/h1.svg')

plt.show()

#save as svg

(0, (array([6.92930200e+03, 1.37947570e+04, 2.05072190e+04, ...,
       6.13748295e+07, 6.14054660e+07, 6.14376831e+07]), array([0.08   , 0.05875, 0.07625, ..., 0.28   , 0.12875, 0.055  ]), '2R', 'DRC Forest (Tshopo)'))
(1, (array([  106585.182 ,   184446.54  ,   221036.9535, ..., 49335563.836 ,
       49342660.191 , 49349478.1905]), array([0.0375 , 0.39875, 0.0975 , ..., 0.1025 , 0.0975 , 0.075  ]), '2L', 'DRC Forest (Tshopo)'))
(2, (array([9.59644600e+03, 1.86722940e+04, 2.45557020e+04, ...,
       5.29666158e+07, 5.30451037e+07, 5.30983105e+07]), array([0.05125, 0.34875, 0.08875, ..., 0.1675 , 0.055  , 0.0725 ]), '3R', 'DRC Forest (Tshopo)'))
(3, (array([   68036.8035,    82886.502 ,   111535.469 , ..., 41931997.7705,
       41943264.259 , 41951070.9215]), array([0.055  , 0.07125, 0.0625 , ..., 0.57   , 0.57   , 0.61625]), '3L', 'DRC Forest (Tshopo)'))
(4, (array([4.78364350e+03, 2.33791065e+04, 3.78083210e+04, ...,
       2.42617551e+07, 2.42873805e+07, 2.43315226e+07]), array([0.5